In [11]:
import os
import cv2
import json
import config
import pathlib
import rasterio
import numpy as np
import pandas as pd
from config import *
import earthpy.plot as ep
import earthpy.spatial as es
from dataset import read_img
from matplotlib import pyplot as plt
import subprocess
import pyperclip

# setup gpu
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

train_df = pd.read_csv(config.train_dir)
test_df =  pd.read_csv(config.test_dir)
valid_df = pd.read_csv(config.valid_dir)
p_train_json = config.p_train_dir
p_test_json = config.p_test_dir
p_valid_json = config.p_valid_dir

In [2]:
print(f"Total number of training images = {len(train_df)}")
print(f"Total number of test images = {len(test_df)}")
print(f"Total number of validation images = {len(valid_df)}")

Total number of training images = 1
Total number of test images = 6
Total number of validation images = 1


In [3]:
def class_balance_check(patchify, data_dir):
    """
    Summary:
        checking class percentage in full dataset
    Arguments:
        patchify (bool): TRUE if want to check class balance for patchify experiments
        data_dir (str): directory where data files are saved 
    Return:
        class percentage
    """
    if patchify:
        with open(data_dir, "r") as j:
            train_data = json.loads(j.read())
        labels = train_data["masks"]
        patch_idx = train_data["patch_idx"]

    # commented out by manik (as patchify is false, it's CFR or CFR_CB, so it's deprecated)
    else:
        train_data = pd.read_csv(data_dir)
        labels = train_data.masks.values
        patch_idx = None

    total = 0
    class_name = {}

    for i in range(len(labels)):
        mask = cv2.imread(labels[i])
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        # mask[mask < 105] = 0
        # mask[mask > 104] = 1
        if patchify:
            idx = patch_idx[i]
            mask = mask[idx[0] : idx[1], idx[2] : idx[3]]

        total_pix = mask.shape[0] * mask.shape[1]
        total += total_pix

        dic = {}
        keys = np.unique(mask)
        for i in keys:
            dic[i] = np.count_nonzero(mask == i)

        for key, value in dic.items():
            if key in class_name.keys():
                #problems
                class_name[key] = value + class_name[key]
            else:
                class_name[key] = value

    for key, val in class_name.items():
        class_name[key] = (val / total) * 100

    print("Class percentage:")
    for key, val in class_name.items():
        print("class pixel: {} = {}".format(key, val))
    print(f"unique value in the mask {class_name.keys()}")

In [7]:
print("class percentage of traning data before patch")
class_balance_check(patchify=False, data_dir=config.train_dir)
print(".........................................................................................")
print("class percentage of traning data after patch")
class_balance_check(patchify=True, data_dir=config.p_train_dir)

class percentage of traning data before patch


[ WARN:0@306.995] global /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp (462) readData OpenCV TIFF: TIFFRGBAImageOK: Sorry, can not handle images with 32-bit samples


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [6]:
def check_height_width(data_dir):
    """
    Summary:
        check unique hight and width of images from dataset
    Arguments:
        data_dir (str): path to csv file
    Return:
        print all the unique height and width
    """

    data = pd.read_csv(data_dir)
    # removing UU or UMM or UM
    # data = data[data['feature_ids'].str.contains('uu_00') == False]
    #problems
    # data = data[data["feature_ids"].str.contains("umm_00") == False]
    # data = data[data["feature_ids"].str.contains("um_00") == False]

    print("Dataset:  ", data.shape)

    input_img = data.feature_ids.values
    input_mask = data.masks.values

    input_img_shape = []
    input_mask_shape = []

    for i in range(len(input_img)):
        img = cv2.imread(input_img[i])
        mask = cv2.imread(input_mask[i])

        if img.shape not in input_img_shape:
            input_img_shape.append(img.shape)

        if mask.shape not in input_mask_shape:
            input_mask_shape.append(mask.shape)

    print("Input image shapes: ", input_img_shape)
    print("Input mask shapes: ", input_mask_shape)

In [8]:
print("Unique height and width of training dataset")
check_height_width(config.train_dir)
print(".........................................................................................")
print("Unique height and width of testing dataset")
check_height_width(config.test_dir)
print(".........................................................................................")

print("Unique height and width of validation dataset")
check_height_width(config.valid_dir)

Unique height and width of training dataset
Dataset:   (1, 2)


[ WARN:0@429.270] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/mnt/hdd2/mdsamiul/project/dataset/rice_data_training/awc00'): can't open/read file: check file path/integrity
[ WARN:0@429.270] global /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp (462) readData OpenCV TIFF: TIFFRGBAImageOK: Sorry, can not handle images with 32-bit samples


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
def return_csv_from_path(csv_path=config.csv_logger_path):
    csv_list = []
    # Iterate through each subdirectory
    for folder in csv_path.iterdir():
        # Check if the entry is a directory
        if folder.is_dir():
            # Iterate through files in the subdirectory
            for file in folder.iterdir():
                # Check if the entry is a file
                if file.is_file():
                    csv_list.append(file)
    # print(csv_list)
    return csv_list
                    

def _plot_from_csv(csv_path, name, x_axis_name, y_axis_name, columns_to_plot=None):
    pathlib.Path((config.root_dir /"logs" / "plots"/"metrics_plots")).mkdir(parents=True, exist_ok=True)
    df = pd.read_csv(csv_path)
    epochs = df['epoch']
    if columns_to_plot is not None:
        columns_to_plot = columns_to_plot
    else:
        columns_to_plot = df.columns.to_list()[1:]

    plt.figure(figsize=(12, 8))
    for column in columns_to_plot:
        plt.plot(epochs, df[column], label=column, linewidth=3.0,
            marker="o",
            markersize=5)

    plt.title(f"{y_axis_name}_over_{x_axis_name}")
    plt.xlabel(x_axis_name)
    plt.ylabel(y_axis_name)
    plt.xticks(epochs.astype(int))
    plt.legend()
    plt.savefig(config.root_dir/"logs"/"plots"/"metrics_plots"/name)
    plt.show()

def plot_metrics_vs_epochs(csv_path, name, x_axis_name= "Epochs", y_axis_name="Metrics_score",columns_to_plot=None):
    _plot_from_csv(csv_path=csv_path, name=name,x_axis_name=x_axis_name, y_axis_name=y_axis_name, columns_to_plot=columns_to_plot)

def plot_metric_vs_epochs_vs_models(metric_name="val_f1_score"):
    pathlib.Path((config.root_dir /"logs"/ "plots"/"csv_for_plotting")).mkdir(parents=True, exist_ok=True)
    csv_list = return_csv_from_path()
    result_df = pd.DataFrame()
    for csv_path in csv_list:
        df = pd.read_csv(csv_path)
        result_df[os.path.basename(csv_path)] = df[metric_name]
    result_df.index.name = "epoch"
    result_df.to_csv(os.path.join(config.root_dir/"logs"/"plots"/"csv_for_plotting"/f"{metric_name}_vs_epoch.csv"), encoding='utf-8',index=True, header=True)
    _plot_from_csv(config.root_dir/"logs"/"plots"/"csv_for_plotting"/f"{metric_name}_vs_epoch.csv", x_axis_name= "Epochs", y_axis_name=metric_name, name=metric_name)

In [ ]:
plot_metrics_vs_epochs(config.csv_logger_path/"unet"/"unet_ex_training_ep_20.csv",name='metrics')
plot_metrics_vs_epochs(config.csv_logger_path/"unet"/"unet_ex_training_ep_20.csv",name='metrics',columns_to_plot=["f1_score"])
plot_metric_vs_epochs_vs_models()
plot_metric_vs_epochs_vs_models(metric_name="recall")

In [ ]:
def display_all(data, name):
    """
    Summary:
        save all images into single figure
    Arguments:
        data : data file holding images path
        directory (str) : path to save images
    Return:
        save images figure into directory
    """

    pathlib.Path((visualization_dir / "display")).mkdir(parents=True, exist_ok=True)
    pathlib.Path((visualization_dir / "display"/"train")).mkdir(parents=True, exist_ok=True)
    pathlib.Path((visualization_dir / "display"/"test")).mkdir(parents=True, exist_ok=True)
    pathlib.Path((visualization_dir / "display"/"valid")).mkdir(parents=True, exist_ok=True)

    for i in range(len(data)):
        image = read_img(data.feature_ids.values[i])
        mask = read_img(data.masks.values[i], label=True)
        id = data.feature_ids.values[i].split("/")[-1]
        display_list = {"image": image, "label": mask}

        plt.figure(figsize=(12, 8))
        title = list(display_list.keys())

        for i in range(len(display_list)):
            plt.subplot(1, len(display_list), i + 1)
            plt.title(title[i])
            plt.imshow((display_list[title[i]]), cmap="gray")
            plt.axis("off")

        prediction_name = "img_id_{}".format(id)  # create file name to save
        plt.savefig(
            os.path.join((visualization_dir / "display"/ name), prediction_name),
            bbox_inches="tight",
            dpi=800,
        )
        plt.clf()
        plt.cla()
        plt.close()

In [ ]:
print("displaying training images and masks")
display_all(data=train_df)

In [ ]:
print("displaying testing images and masks")
display_all(data=test_df)

In [ ]:
print("displaying validation images and masks")
display_all(data=valid_df)

In [9]:
eval_csv = pd.read_csv("/mnt/hdd2/mdsamiul/project/imseg_csml/data/csv/train.csv")
masks = eval_csv["feature_ids"].to_list()
ext = ["_vv.tif","_vh.tif","_nasadem.tif"]
masks = masks[0]
masks= [masks+ex for ex in ext]
print(masks)
for p in masks:
    with rasterio.open(p) as im:
        image = im.read(1)
        print("...............................")
        # print(p)
        # print(np.unique(image, return_counts=False))
        print(np.mean(image))
        print(np.std(image))
        print("...............................")

['/mnt/hdd2/mdsamiul/project/dataset/rice_data_training/awc00_vv.tif', '/mnt/hdd2/mdsamiul/project/dataset/rice_data_training/awc00_vh.tif', '/mnt/hdd2/mdsamiul/project/dataset/rice_data_training/awc00_nasadem.tif']
...............................
-14.243971
4.883937
...............................
...............................
-22.869745
6.5032916
...............................
...............................
-14.243971
4.883937
...............................


In [ ]:
def rename_files(datapath):
    # List all files in the directory
    files = os.listdir(datapath)
    
    for filename in files:
        # Extract the file extension
        _, ext = os.path.splitext(filename)
        
        # Check if the filename starts with DEM_ab.tif
        if filename.startswith("DEM_"):
            new_filename = filename.replace("DEM_", "").replace(".tif", "_nasadem.tif")
        
        # Check if the filename starts with VV_ab.tif
        elif filename.startswith("VV_"):
            new_filename = filename.replace("VV_", "").replace(".tif", "_vv.tif")
        
        # Check if the filename starts with VH_ab.tif
        elif filename.startswith("VH_"):
            new_filename = filename.replace("VH_", "").replace(".tif", "_vh.tif")
        
        # Check if the filename starts with GT_ab.tif
        elif filename.startswith("GT_"):
            new_filename = filename.replace("GT_", "")
        
        else:
            # If none of the conditions are met, skip this file
            raise ValueError("files_name_mismatch")
        
        # Construct the new filepath
        new_filepath = os.path.join(datapath, new_filename)
        
        # Rename the file
        os.rename(os.path.join(datapath, filename), new_filepath)
        print(f"Renamed {filename} to {new_filename}")


In [ ]:
datapath = config.dataset_dir
rename_files(datapath)

In [13]:
# Run the command in the terminal
command = "python visualization.py"
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# Get the terminal output
terminal_output = result.stdout

# Save the output to an RTF file
rtf_filename = "data_statistics.rtf"
with open(rtf_filename, "w") as rtf_file:
    # rtf_file.write("{\\rtf1\\ansi\n")
    rtf_file.write(terminal_output)
    # rtf_file.write("}")

print(f"Terminal output saved to {rtf_filename}")

Terminal output saved to data_statistics.rtf
